In [1]:
data_dir = "/data/projects/dschaub/ANCA-GN_transcriptomics/data/single-cell/exploratory"
working_dir = "/data/projects/dschaub/ANCA-GN_transcriptomics"

In [2]:
%load_ext autoreload
%autoreload 2

import sys
import os
os.chdir(working_dir)

import anndata as ad
import matplotlib.pyplot as plt
import mudata as md
import muon as mu
import pandas as pd
import numpy as np
import scanpy as sc
from scipy import sparse
import matplotlib as mpl

from utils.utils import *
from utils.plotting import *

sc.settings.verbosity = 0
sc.set_figure_params(dpi=80)
sns.set(style="white")

Global seed set to 0
/opt/conda/envs/scvi-env/lib/python3.9/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/opt/conda/envs/scvi-env/lib/python3.9/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


## Load data

In [3]:
path = os.path.join(
    data_dir, "ANCA_exploratory_27PK27PB_Tcells_HarmonyR_annotated.h5mu"
)
mudata = md.read_h5mu(path)
mod_rna = mudata.mod["rna"]
mod_cite = mudata.mod["cite"]
mudata

MuData object with n_obs × n_vars = 72416 × 21625
  2 modalities
    rna:	72416 x 21419
      obs:	'patient', 'sample', 'tissue', 'cell_type_yu', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 'leiden_0.6', 'leiden_0.7', 'leiden_0.8', 'leiden_0.9', 'leiden_1.0', 'leiden_1.1', 'leiden_1.2', 'leiden_1.3', 'leiden_1.4', 'leiden_1.5', 'leiden_1.6', 'leiden_1.7', 'leiden_1.8', 'leiden_1.9', 'cell_type', 'cell_type_v1'
      var:	'n_cells', 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'mean', 'std'
      uns:	'cell_type_colors', 'cell_type_v1_colors', 'cell_type_yu_colors', 'leiden', 'leiden_0.3_colors', 'leiden_0.4_colors', 'leiden_0.5_colors', 'leiden_0.6_colors', 'leiden_0.7_colors', 'leiden_0.8_colors', 'leiden_0.9_colors', 'leiden_1.0_colors', 'leiden_1.1_colors', 'leiden_1.2_colors', 'leiden_1.3_colors', 'leiden_1.4_colors', 'leiden_1.5_colors', 'leiden_1.6_colors', 'leiden_1.7_colors', 'leiden_1.8_colors', 'leiden_1.9_colors', 'log1p', 'neighbors', 'patient_colors', 'sample_colors', 'tissue_colors', 'umap'
      obsm:	'X_harmony', 'X_umap'
      layers:	'counts', 'log1p', 'scaled'
      obsp:	'connectivities', 'distances'
    cite:	72416 x 206
      obs:	'patient', 'sample', 'tissue', 'cell_type_yu', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts'
      var:	'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'mean', 'std'
      layers:	'clr', 'counts', 'scaled'

## Subset CD8EMRM

In [4]:
celltype_key = "cell_type"
mudata = mudata[mudata.mod["rna"].obs[celltype_key] == "CD8+ EM/RM"].copy()
mod_rna = mudata.mod["rna"]
mod_cite = mudata.mod["cite"]

## Preprocess data

In [ ]:
print(mod_rna.shape)
sc.pp.filter_genes(mod_rna, min_cells=10)
print(mod_rna.shape)

In [ ]:
# remove any patient that contains less than 2 cells
cells_per_patient = mod_rna.obs["patient"].value_counts()
patients_to_remove = cells_per_patient[cells_per_patient < 2].index.tolist()
mod_rna = mod_rna[~mod_rna.obs["patient"].isin(patients_to_remove)].copy()
mod_cite = mod_cite[mod_rna.obs_names, :].copy()

### Quality control

In [ ]:
plot_qc(mod_rna, celltype_key)

In [ ]:
# number of unique patients
len(mod_rna.obs["patient"].unique())

## Integrate data

In [ ]:
log_normalize(mod_rna)
protein_clr(mod_cite)

In [ ]:
model, rna_subset = integrate_with_totalvi(
    mod_rna,
    mod_cite,
    batch_key="patient",
    n_top_genes=4000,
    # continuous_covariate_keys=["pct_counts_mt", "total_counts"],
    # continuous_covariate_keys=["total_counts"],
    # continuous_covariate_batch_keys=["pct_counts_mt"],
    empirical_protein_background_prior=True,
)

In [ ]:
plt.plot(model.history["elbo_train"], label="train")
plt.plot(model.history["elbo_validation"], label="val")
plt.title("Negative ELBO over training epochs")
# plt.ylim(500, 1500)
plt.legend()

In [ ]:
rep_name = "X_totalvi"
mod_rna.obsm[rep_name] = model.get_latent_representation()
sc.pp.neighbors(mod_rna, use_rep=rep_name)
sc.tl.umap(mod_rna)

In [ ]:
sc.pl.umap(
    mod_rna,
    color=["sample", "patient", "tissue"],
    # return_fig=True,
    # legend_loc="on data",
    # save="_leiden.png",
    wspace=0.8,
)

In [ ]:
resolutions = np.linspace(0.4, 2.0, 17)
for res in resolutions:
    res = np.round(res, 1)
    print(f"Running Leiden clustering with resolution {res}")
    sc.tl.leiden(mod_rna, key_added=f"leiden_{res}", resolution=res)

In [ ]:
plot_leiden_results(mod_rna, rep_name="")

## Save data

In [ ]:
save_path = os.path.join(
    data_dir, "ANCA_exploratory_27PK27PB_CD8Teff_TotalVI.h5mu"

)
mudata = md.MuData(
    {
        "rna": mod_rna.copy(),
        "cite": mod_cite.copy(),
    }
)
mudata.write_h5mu(save_path)